In [1]:
import os
# Change native directory to root
os.chdir(os.path.dirname(os.getcwd()))

In [9]:
import glob
import pandas as pd
import numpy as np
import random
# random seed for reproducibility
random.seed(123)

In [3]:
meta_data = pd.read_csv('data/EANLIJST_METADATA.csv', sep=';')
meta_data

,EAN_ID,Patrimonium Type,Patrimonium Functietype,Meetmethode,Richting
0,681,NaN,NaN,Continu,Afname
1,125,NaN,NaN,Continu,Afname
2,213,NaN,NaN,Continu,Afname
3,603,NaN,NaN,Continu,Afname
4,1699,NaN,NaN,Continu,Afname
...,...,...,...,...,...
1736,963,Gebouw,Sportcomplex,Continu,Afname
1737,1364,NaN,NaN,Continu,Afname
1738,473,Gebouw,Werkplaats,Continu,Afname
1739,965,Gebouw,Administratief centrum,Continu,Afname


In [12]:
build_list = list(meta_data['EAN_ID'].unique())
test_list = random.sample(build_list, int(len(build_list) * 0.1))
train_list = list(set(build_list) - set(test_list))

In [6]:
exam = pd.read_csv('data/buildings/1032.csv')
exam

,ds,Power,function
0,2019-01-01 00:15:00,43.33,Zwembad
1,2019-01-01 00:30:00,42.60,Zwembad
2,2019-01-01 00:45:00,44.69,Zwembad
3,2019-01-01 01:00:00,44.17,Zwembad
4,2019-01-01 01:15:00,46.16,Zwembad
...,...,...,...
105211,2021-12-31 23:00:00,62.88,Zwembad
105212,2021-12-31 23:15:00,59.95,Zwembad
105213,2021-12-31 23:30:00,60.99,Zwembad
105214,2021-12-31 23:45:00,59.11,Zwembad


In [48]:
meta_data[meta_data['Patrimonium Functietype'] == 'Zwembad']

,EAN_ID,Patrimonium Type,Patrimonium Functietype,Meetmethode,Richting
5,452,Gebouw,Zwembad,Continu,Afname
18,1383,Gebouw,Zwembad,Continu,Afname
20,1496,Gebouw,Zwembad,Continu,Afname
46,1032,Gebouw,Zwembad,Continu,Afname
59,1151,Gebouw,Zwembad,Continu,Afname
104,45,Gebouw,Zwembad,Continu,Afname
183,1647,Gebouw,Zwembad,Continu,Afname
196,376,Gebouw,Zwembad,Continu,Afname
199,775,Gebouw,Zwembad,Continu,Afname
218,979,Gebouw,Zwembad,Continu,Afname


In [22]:
#file = build_list[0]
output_path='data/full.csv'
build_list = meta_data[meta_data['Patrimonium Functietype'].notna()].EAN_ID.unique()
for ID in build_list: #build_list
    pathfile = 'data/buildings/'+ str(ID) + '.csv'
    ts = pd.read_csv(pathfile, index_col='ds', parse_dates=['ds'])
    ts.index.name = 'datetime'
    ts.drop_duplicates(inplace=True)
    # resample power to hourly
    ts.Power = ts.Power.resample('1H').sum()
    # convert power to energy
    ts.kWh = ts.Power * 0.25
    ts['month'] = ts.index.month
    ts['weekday'] = ts.index.weekday
    ts['hour'] = ts.index.hour
    #ts = pd.concat([ts, weather], axis=1, join='inner')
    ts['yearly_cons'] = ts.kWh.resample("1Y").sum().max()
    ts['Target'] = np.log1p(ts.kWh)
    ts['Target'] = ts.Target / ts.Target.resample("1Y").sum().max()
    ts['Target-1'] = ts['Target'].shift(freq='1D').dropna()
    ts['Target-7'] = ts['Target'].shift(freq='7D').dropna()
    ts['ID'] = ID
    ts['type'] = meta_data[meta_data['EAN_ID'] == ID]['Patrimonium Type'].values[0]
    ts['function'] = meta_data[meta_data['EAN_ID'] == ID]['Patrimonium Functietype'].values[0]
    ts_small, nans = reduce_mem_usage(ts)
    ts_small['datetime'] = ts_small.index
    ts_small.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)

/tmp/ipykernel_172049/3739510695.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ts.kWh = ts.Power * 0.25
/tmp/ipykernel_172049/3739510695.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ts.kWh = ts.Power * 0.25


Memory usage of properties dataframe is : 0.2874755859375  MB
******************************
Column:  Power
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  month
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  weekday
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  hour
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  yearly_cons
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Target
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Target-1
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Target-7
dtype before:  float64
dtype after: 

/tmp/ipykernel_172049/3739510695.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ts.kWh = ts.Power * 0.25
/tmp/ipykernel_172049/3739510695.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ts.kWh = ts.Power * 0.25


Memory usage of properties dataframe is : 0.019500732421875  MB
******************************
Column:  Power
dtype before:  float64
dtype after:  uint16
******************************
******************************
Column:  month
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  weekday
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  hour
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  yearly_cons
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Target
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Target-1
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Target-7
dtype before:  float64
dtype after:

/tmp/ipykernel_172049/3739510695.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ts.kWh = ts.Power * 0.25
/tmp/ipykernel_172049/3739510695.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ts.kWh = ts.Power * 0.25


...

/tmp/ipykernel_172049/3739510695.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ts.kWh = ts.Power * 0.25
/tmp/ipykernel_172049/3739510695.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ts.kWh = ts.Power * 0.25


Memory usage of properties dataframe is : 0.01483154296875  MB
******************************
Column:  Power
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  month
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  weekday
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  hour
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  yearly_cons
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Target
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Target-1
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Target-7
dtype before:  float64
dtype after:

/tmp/ipykernel_172049/3739510695.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ts.kWh = ts.Power * 0.25
/tmp/ipykernel_172049/3739510695.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ts.kWh = ts.Power * 0.25


Memory usage of properties dataframe is : 0.107940673828125  MB
******************************
Column:  Power
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  month
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  weekday
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  hour
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  yearly_cons
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Target
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Target-1
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  Target-7
dtype before:  float64
dtype after

/tmp/ipykernel_172049/3739510695.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ts.kWh = ts.Power * 0.25


In [25]:
output_path='data/'
data = pd.read_csv(output_path+'full.csv')
# split strings in a list
ts_small = data[data['ID'].isin(train_list)]
ts_small.to_csv(output_path+'full_train.csv',index=False)
ts_small = data[data['ID'].isin(test_list)]
ts_small.to_csv(output_path+'full_test.csv', index=False)

In [8]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist